In [1]:
import pixiedust

Pixiedust database opened successfully


In [2]:
import random

In [3]:
import pandas as pd
import math

In [4]:
def open_tabel(file):
    tabel_alternatif = pd.read_csv(file, index_col=0)
    
    return tabel_alternatif

In [5]:
def add_parameter(nilai, tabel_alternatif):
    tabel_alternatif["Parameter"] = nilai
    
    return tabel_alternatif

In [6]:
def get_jurusan(tabel_alternatif):
    pilihan_jurusan = []
    for column in tabel_alternatif:
        if column != "Parameter":
            pilihan_jurusan.append(column)
            
    return pilihan_jurusan

In [7]:
def deviasi(jurusan, jurusan_compare, index_name):
    nilai_deviasi = tabel_alternatif[jurusan][index_name] - tabel_alternatif[jurusan_compare][index_name]
    
    return nilai_deviasi

In [8]:
def idx_pref_global1(fp_gaussian, column_test, weight):
    index_preferensi_global = {}
    
    for column in fp_gaussian:
        nilai_gaussian = 0
        for kriteria in fp_gaussian.index:
        
            if column_test.casefold() == column.casefold().split("&")[0].replace(" (ips)", "")\
            or column_test.casefold() == column.casefold().split("&")[0].replace(" (ipa)", ""):
                nilai_gaussian += fp_gaussian[column][kriteria]
            else:
                nilai_gaussian += fp_gaussian[column][kriteria]*weight[kriteria]
            
        index_preferensi_global[column] = nilai_gaussian
        
    tabel_index_preferensi_global = pd.DataFrame(index_preferensi_global, index = ["Index Preferensi Global"])

    return tabel_index_preferensi_global

In [9]:
def idx_pref_global(fp_gaussian, weigth):
    index_preferensi_global = {}
    
    for column in fp_gaussian:
        nilai_gaussian = 0
        for kriteria in fp_gaussian.index:
            nilai_gaussian += fp_gaussian[column][kriteria]*weight[kriteria]
            
        index_preferensi_global[column] = nilai_gaussian
        
    tabel_index_preferensi_global = pd.DataFrame(index_preferensi_global, index = ["Index Preferensi Global"])

    return tabel_index_preferensi_global

In [10]:
def entering_flow(pilihan_jurusan, index_preferensi_global):
    e_flow = {}

    for item in pilihan_jurusan:
        ef = 0
        for piece in index_preferensi_global:
            s = piece.split("&")

            if item in s and s.index(item) == 1:
                ef += index_preferensi_global[piece]["Index Preferensi Global"] * (1/(len(pilihan_jurusan) - 1))

        e_flow[item] = ef

    return e_flow

In [11]:
def leaving_flow(pilihan_jurusan, index_preferensi_global):
    l_flow = {}

    for item in pilihan_jurusan:
        lf = 0
        for piece in index_preferensi_global:
            s = piece.split("&")

            if item in s and s.index(item) == 0:
                lf += index_preferensi_global[piece]["Index Preferensi Global"] * (1/(len(pilihan_jurusan) - 1))

        l_flow[item] = lf

    return l_flow

In [12]:
def net_flow(pilihan_jurusan, e_flow, l_flow):
    n_flow = {}

    for jurusan in pilihan_jurusan:
        n_flow[jurusan] = l_flow[jurusan] - e_flow[jurusan]

    return n_flow

In [13]:
def get_std(tabel_alternatif):
    #nyari simpangan baku
    std = tabel_alternatif.std(axis=1)

    return std

In [14]:
def get_mean(tabel_alternatif):
    #nyari rata2
    mean = tabel_alternatif.mean(axis=1)

    return mean

In [15]:
def get_params(mean, nilai, std):
    params = []
    for bjg in range(len(nilai)):
        tot = (nilai[bjg]- mean[bjg]) / std[bjg]
        params.append(tot)

    return params

In [16]:
def fungsi_preferensi_gaussian(tabel_alternatif, q, p):
    fp_gaussian = {}
    fp_gaussian_val = list()
    tabel_gaus = pd.DataFrame()
    #adaw = {}

    for jurusan in pilihan_jurusan:
        for jurusan_compare in pilihan_jurusan:
            if jurusan == jurusan_compare:
                continue
            else:
                for index_name in tabel_alternatif.index:
                    if "(IPA)" in jurusan.split():
                        if index_name == "Matematika" or index_name == "Inggris" or index_name == "Indonesia" or index_name == "Fisika" or index_name == "Kimia" or index_name == "Biologi":

                               #menghitung deviasi
                            nilai_deviasi = deviasi(jurusan, jurusan_compare, index_name)

                            #menghitung fungsi preferensi gaussian
                            if nilai_deviasi <= q:
#                                 gaussian = 1 - math.exp(-1*(((nilai_deviasi)**2) / (2*((tabel_alternatif["Parameter"][index_name])**2))))
                                gaussian = 0
                            else:
                                gaussian = 1
                                #adaw[f"{jurusan}&{jurusan_compare} ({index_name})"] = nilai_deviasi

                            deviasi_name = f"{jurusan}&{jurusan_compare}"
                            fp_gaussian_val.append(gaussian)
                        else:
                            fp_gaussian_val.append(0)

                    elif "(IPS)" in jurusan.split():
                        if index_name == "Ekonomi" or index_name == "Inggris" or index_name == "Indonesia" or index_name == "Geografi" or index_name == "Sosiologi":

                            #menghitung deviasi
                            nilai_deviasi = deviasi(jurusan, jurusan_compare, index_name)

                            #menghitung fungsi preferensi gaussian
                            if nilai_deviasi <= q:
#                                 gaussian = 1 - math.exp(-1*(((nilai_deviasi)**2) / (2*((tabel_alternatif["Parameter"][index_name])**2))))
                                gaussian = 0
                            else:
                                gaussian = 1
                                #adaw[f"{jurusan}&{jurusan_compare} ({index_name})"] = nilai_deviasi

                            deviasi_name = f"{jurusan}&{jurusan_compare}"
                            fp_gaussian_val.append(gaussian)
                        else:
                            fp_gaussian_val.append(0)

                    try:
                        fp_gaussian[deviasi_name] = fp_gaussian_val
                        
                    except NameError:
                        continue
                        
            if tabel_gaus.empty:
                tabel_gaus = pd.DataFrame(fp_gaussian,[
                    "Matematika",
                    "Inggris",
                    "Indonesia",
                    "Fisika",
                    "Kimia",
                    "Biologi",
                    "Ekonomi",
                    "Geografi",
                    "Sosiologi",
                ])
            else:
                try:
                    tabel_gaus[deviasi_name] = fp_gaussian_val
                except ValueError:
                    continue
            fp_gaussian_val  = list()

    return tabel_gaus